# 基础函数

In [8]:
import cv2

img = cv2.imread("2.jpg")
cv2.imshow("img", img)

# 二值化
th_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
cv2.imshow("th_img", th_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 截取图像

In [3]:
import cv2
import numpy as np

import cv2

# 定义回调函数，用于处理鼠标事件
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at ({x}, {y})")
        param['clicks'] += 1
        if param['clicks'] == 4:
            print("Four clicks reached. Exiting...")
            cv2.destroyAllWindows()

def extract_quadrilateral(image, vertices):
    # 定义平行四边形的四个顶点
    pts_src = np.array(vertices, dtype=np.float32)

    # 定义输出图像中矩形的四个顶点
    height, width = image.shape[:2]
    pts_dst = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # 计算透视变换矩阵
    M = cv2.getPerspectiveTransform(pts_src, pts_dst)

    # 应用透视变换
    result = cv2.warpPerspective(image, M, (width, height))

    return result

# 读取图像
image = cv2.imread('2.jpg')
global q_loc

# 创建窗口并设置鼠标回调函数
cv2.namedWindow('Image')
clicks_count = {'clicks': 0}
cv2.setMouseCallback('Image', mouse_callback, clicks_count)

while True:
    # 显示图像
    cv2.imshow('Image', image)

    # 等待用户按键，若按下ESC键则退出循环
    key = cv2.waitKey(1)
    if key == 27:
        break

# 定义平行四边形的四个顶点坐标（顺时针或逆时针定义）
quadrilateral_vertices = q_loc

# 截取平行四边形
result_image = extract_quadrilateral(image, quadrilateral_vertices)

# 显示原始图像和截取后的图像
cv2.imshow('Original Image', image)
cv2.imshow('Extracted Quadrilateral', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\Build\OpenCV\opencv-4.5.5\modules\imgproc\src\imgwarp.cpp:3392: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'cv::getPerspectiveTransform'


In [7]:
import cv2
import numpy as np

import cv2

# 定义回调函数，用于处理鼠标事件
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at ({x}, {y})")
        q_loc.append([x, y])
        param['clicks'] += 1
        if param['clicks'] == 4:
            print("Four clicks reached. Exiting...")
            cv2.destroyAllWindows()

def extract_quadrilateral(image, vertices):
    # 定义平行四边形的四个顶点
    pts_src = np.array(vertices, dtype=np.float32)

    # 定义输出图像中矩形的四个顶点
    height, width = image.shape[:2]
    print(type(height))
    pts_dst = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # 将图片截取出来
    # 计算透视变换矩阵
    M = cv2.getPerspectiveTransform(pts_src, pts_dst)
    
    # 应用透视变换
    result = cv2.warpPerspective(image, M, (width, height))

    return result

# 读取图像
image = cv2.imread('2.jpg')
global q_loc

# 创建窗口并设置鼠标回调函数
cv2.namedWindow('Image')
clicks_count = {'clicks': 0}
cv2.setMouseCallback('Image', mouse_callback, clicks_count)

while True:
    # 显示图像
    cv2.imshow('Image', image)

    # 等待用户按键，若按下ESC键则退出循环
    key = cv2.waitKey(1)
    if key == 27:
        break

# 定义平行四边形的四个顶点坐标（顺时针或逆时针定义）
quadrilateral_vertices = q_loc

# 截取平行四边形
result_image = extract_quadrilateral(image, quadrilateral_vertices)

# 显示原始图像和截取后的图像
# cv2.imshow('Original Image', image)
cv2.imshow('Extracted Quadrilateral', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Clicked at (1, 604)
Clicked at (1278, 14)
Clicked at (1278, 189)
Clicked at (1, 763)
Four clicks reached. Exiting...
<class 'int'>


error: OpenCV(4.5.5) D:\Build\OpenCV\opencv-4.5.5\modules\imgproc\src\imgwarp.cpp:3392: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'cv::getPerspectiveTransform'


## 找到管道的四个端点

In [2]:
import cv2
import numpy as np

def find_pipe_endpoints(image_path):
    # 读取图像
    image = cv2.imread(image_path)

    # 进行边缘检测
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    
    # 先膨胀后腐蚀
    kernel = np.ones((5, 5), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=2)
    edges = cv2.erode(edges, kernel, iterations=2)
    
    
    # 查看边缘检测结果
    cv2.imshow('Edges', edges)
    
    # 寻找轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 选择最大的轮廓
    largest_contour = max(contours, key=cv2.contourArea)
    
    # 显示最大的轮廓
    cv2.drawContours(image, [largest_contour], -1, (0, 255, 0), 3)
    cv2.imshow('Largest Contour', image)

    # 进行透视变换
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    # 提取四个端点
    if len(approx) == 4:
        pipe_endpoints = approx.reshape(-1, 2)
        return pipe_endpoints
    else:
        print("未能找到四个端点")

# 使用示例
image_path = '1.jpg'  # 替换为实际图像路径
endpoints = find_pipe_endpoints(image_path)
cv2.waitKey(0)
cv2.destroyAllWindows()

if endpoints is not None:
    print("管道四个端点的位置：", endpoints)
else:
    print("未能找到四个端点")

未能找到四个端点
未能找到四个端点


# 网上代码研究

In [3]:
import cv2

# 定义回调函数，用于处理鼠标事件
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at ({x}, {y})")

# 读取图像
image = cv2.imread('2.jpg')

# 创建窗口并设置鼠标回调函数
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', mouse_callback)

while True:
    # 显示图像
    cv2.imshow('Image', image)

    # 等待用户按键，若按下ESC键则退出循环
    key = cv2.waitKey(1)
    if key == 27:
        break

# 关闭窗口
cv2.waitKey(0)
cv2.destroyAllWindows()

Clicked at (2, 605)
Clicked at (0, 764)
Clicked at (1279, 17)
Clicked at (1279, 188)
